In [53]:
import pymongo
import os,csv,sys
import openslide
import dsa_mongo_common_functions as dsa
import cdsa_loader_helper_functions as cdsa_helpers
import pprint
client = pymongo.MongoClient('localhost',27017)
dsa_load_errors_db = client['DSA_LoadErrors']


In [54]:
reload(dsa)
reload(cdsa_helpers)


<module 'cdsa_loader_helper_functions' from 'cdsa_loader_helper_functions.pyc'>

In [55]:
## This is specific to a given fle system and or structurme
slide_root = '/GLOBAL_SCRATCH/ADRC/'  ##Base Path for Slides
dsa_slide_db = client['ADRC_DSA_Slide_DB']  ### These need to be configured for the specific project

### To generalize this, need to describe organization, most common will be  PATIENT/STAIN_TYPE as subdirectories
subj_dir_list = [x for x in os.listdir(slide_root) if os.path.isdir(os.path.join(slide_root,x))]
print len(subj_dir_list),"Potential Patient directories were identified"

103 Potential Patient directories were identified


In [19]:
def find_rawslide_lists( slide_root_path ):
        """project_name is passed along with the potentially more than one root image path for ndpi files"""
        slide_files = []

        slide_root_path  = slide_root_path.rstrip('/')
        print slide_root_path
        for dpath, dnames, fnames in os.walk( slide_root_path, followlinks=True):
                
                for file in fnames:
                    if '.ndpi' in file or '.svs' in file:
                                slide_files.append(dpath +'/'+file)
        print len(slide_files),"SVS or NDPI files were located"
        return slide_files

In [20]:
print subj_dir_list

['ADRC61-124', 'ADRC50-10', 'ADRC61-128', 'ADRC50-40', 'ADRC51-60', '@eaDir', 'ADRC60-150', 'ADRC61-120', 'ADRC57-85', 'ADRC55-200', 'ADRC49-07', 'ADRC50-11', 'ADRC51-2', 'ADRC51-134', '.ipynb_checkpoints', 'SamMarch20', 'ADRC61-139', 'ADRC61-125', 'ADRC53-300', 'ADRC56-112', 'ADRC43-24', 'ADRC61-73', 'ADRC62-74', 'ADRC61-84', 'ADRC62-14', 'ADRC49-04', 'ADRC56-97', 'ADRC51-06', 'ADRC36-04', 'ADRC59-164', 'ADRC56-18', 'ADRC59-81', 'ADRC62-110', 'ADRC61-143', 'ADRC47-60', 'ADRC62-6', 'ADRC61-97', 'ADRC62-64', 'ADRC54-125', 'newcerebptstoload', 'ADRC61-71', 'ADRC60-110', 'ADRC61-20', 'ADRC56-06', 'ADRC53-55', 'ADRC59-8', 'ADRC60-137', 'ADRC61-70', 'ADRC61-75', 'ADRC55-100', 'ADRC56-160', 'ADRC51-10', 'ADRC39-44', 'ADRC58-146', 'ADRC60-88', 'ADRC61-41', 'ADRC62-62', 'ADRC61-112', 'ADRC55-194', 'ADRC60-83', 'ADRC50-41', 'ADRC61-60', 'ADRC62-32', 'ADRC54-76', 'NickSlidesFeb20', 'ADRC59-35', 'ADRC47-61', 'ADRC55-108', 'ADRC61-83', 'ADRC51-94', 'ADRC59-91', 'ADRC58-119', 'ADRC53-95', 'ADRC53-3

In [56]:
### So this creates a document WITHIN the current database to store raw information about the slides
## Now that I think about this, I really should not store the filename but the File Hash as I.. want to change the filenames

for sd in subj_dir_list:
    print sd,"is being processed",
    curr_svs_slide_list = find_rawslide_lists(  os.path.join(slide_root,sd)  )
    slides_processed = newly_processed = dup_slide = rescanned_slides =  0
    for sld in curr_svs_slide_list:

        slide_name = os.path.basename(sld)
        qry = dsa_slide_db['RawSlideData'].find_one( {'slide_name':slide_name})
        #print qry
        if not qry:
            fs = os.path.getsize(sld)
            #md5Checksum = dsa.md5sum(sld)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, sld_properties) = cdsa_helpers.openslide_test_file_mongo( sld, 'ndpi', client)
            if openslide_could_open:
                prep_type = 'Unknown'
                slide_metadata = { 'slide_w_path': sld, 'slide_name': slide_name, 'file_size':fs, 'width':width, 'height':height,
                                 'orig_resolution': orig_resolution, 'sld_properties': cdsa_helpers.clean_openslide_keys ( sld_properties), 'slide_md5': md5, 'prep_type': prep_type
                                 }
                dsa_slide_db['RawSlideData'].insert_one(slide_metadata)
                newly_processed +=1 
            else:
                print "UNABLE TO OPEN FILE??",sld
                ###Need to flag/load this in to an error database

        else:
            fs = os.path.getsize(sld)
            ## Double check if file size matches
#             if qry['file_size'] != fs:
#                 #print "File size mismatch??",fs,qry['file_size'],qry['slide_w_path'],sld
#                 load_errors_db['rescanned_slides'].insert_one( {'loaded_slide': qry['slide_w_path'], 'rescanned_slide': sld}             )
#                 rescanned_slides +=1 
#             else:
#                 dup_slide +=1

    
        slides_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d  RESCANNED Slides %d" % (slides_processed, newly_processed, dup_slide, rescanned_slides )
        dsa.LinePrinter(output)


ADRC61-124 is being processed /GLOBAL_SCRATCH/ADRC/ADRC61-124
106 SVS or NDPI files were located
Total Processed: 106  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC50-10 is being processed /GLOBAL_SCRATCH/ADRC/ADRC50-10
58 SVS or NDPI files were located
Total Processed: 58  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC61-128 is being processed /GLOBAL_SCRATCH/ADRC/ADRC61-128
99 SVS or NDPI files were located
Total Processed: 99  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC50-40 is being processed /GLOBAL_SCRATCH/ADRC/ADRC50-40
59 SVS or NDPI files were located
Total Processed: 59  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC51-60 is being processed /GLOBAL_SCRATCH/ADRC/ADRC51-60
10 SVS or NDPI files were located
Total Processed: 10  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0@eaDir is being processed /GLOBAL_SCRATCH/ADRC/@eaDir
0 SVS or NDPI f

Verify failed with: (u"Can't validate JPEG for directory 0: Expected marker at 4294972165, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC61-84/Tau/ADRC61-84_A15_US-tau.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC61-84/Tau/ADRC61-84_A15_US-tau.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC61-84/Tau/ADRC61-84_A15_US-tau.ndpi
Total Processed: 104  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC62-14 is being processed /GLOBAL_SCRATCH/ADRC/ADRC62-14
1 SVS or NDPI files were located
Total Processed: 1  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC60-179 is being processed /GLOBAL_SCRATCH/ADRC/ADRC60-179
96 SVS or NDPI files were located
Total Processed: 96  Newly Processed: 96 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC49-04 is being processed /GLOBAL_SCRATCH/ADRC/ADRC49-04
18 SVS or NDPI files were located
Total Processed: 18  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC56-97 is being processed /GLOBAL_SCRATCH/ADRC/ADRC56-97
116 SVS or NDPI f

 (u"Can't validate JPEG for directory 0: Expected marker at 4294972345, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
Total Processed: 27  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
Total Processed: 37  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/To_Be_Sorted/ADRC51-06_8_S-M.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/To_Be_Sorted/ADRC51-06_8_S-M.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC51-06/To_Be_Sorted/ADRC5

...

 (u"Can't validate JPEG for directory 0: Expected marker at 4294971812, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC62-120/Uncategorized/ADRC62-120_A4_blank.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC62-120/Uncategorized/ADRC62-120_A4_blank.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC62-120/Uncategorized/ADRC62-120_A4_blank.ndpi
Total Processed: 51  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0

 (u"Can't validate JPEG for directory 0: Expected marker at 4294972165, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC62-120/Biels/ADRC62-120_A3_BIELS.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC62-120/Biels/ADRC62-120_A3_BIELS.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC62-120/Biels/ADRC62-120_A3_BIELS.ndpi
Total Processed: 106  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC54-155 is being processed /GLOBAL_SCRATCH/ADRC/ADRC54-155
25 SVS or NDPI files were located
Total Processed: 25  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0Miscellaneous is being processed /GLOBAL_SCRATCH/ADRC/Miscellaneous
31 SVS or NDPI files were located
Total Processed: 31  Newly Processed: 27 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC60-129 is being processed /GLOBAL_SCRATCH/ADRC/ADRC60-129
112 SVS or NDPI files were located
Total Processed: 112  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC53-138 is being processed /GLOBAL_SCRATCH/ADRC/ADRC53

 (u"Can't validate JPEG for directory 0: Expected marker at 4294972345, found none",)


In [49]:
#dsa_slide_db['DSA_Slide_Data'].delete_many({})
#print dsa_slide_db['DSA_Slide_Data'].count()

0


In [ ]:
### I now want to reformat all of this data to make it more useful for DSA ...

In [57]:
all_slides = dsa_slide_db['RawSlideData'].find()

## Since this is pretty easy to recreate, I'm going to empty the current database

dsa_slide_db['DSA_Slide_Data'].delete_many({})

for s in all_slides:
    slide_dict = {}
    pt_id = s['slide_w_path'].split('/')[-3]
    stain_type = s['slide_w_path'].split('/')[-2]
    slide_dict = s
    slide_dict.pop('_id',None)
    
    slide_dict['pt_id'] = pt_id
    slide_dict['stain_type'] = stain_type
    ### Obfuscating the global file path so everything is relative to some base path for the archive/
    slide_dict['thumbnail_image'] = '/thumbnail/' + s['slide_w_path'].replace('/GLOBAL_SCRATCH/ADRC/','')
    slide_dict['slide_w_path'] = '/DZIMS/' + s['slide_w_path'].replace('/GLOBAL_SCRATCH/ADRC/','')+'.dzi'
    dsa_slide_db['DSA_Slide_Data'].insert_one(slide_dict)

    #       var html = sld_info.slide_name + '#' + sld_info.slide_w_path+ '#' + sld_info.slide_name + '#' + sld_info.thumbnail_image;
##    viewer.open('/DZIMS/ADRC50-21/pTDP/OS00-21_14_pTDP_1to10k.ndpi.dzi')
###      var html = sld_info.slide_name + '#' + sld_info.slide_w_path+ '#' + sld_info.slide_name + '#' + sld_info.thumbnail_image;


In [36]:
print dsa_slide_db['DSA_Slide_Data'].distinct('pt_id')

[u'ADRC61-124', u'ADRC61-112', u'ADRC50-10', u'ADRC61-128', u'ADRC50-40', u'ADRC51-60', u'ADRC60-150', u'ADRC61-120', u'ADRC57-85', u'ADRC55-200', u'ADRC49-07', u'ADRC50-11', u'ADRC51-2', u'ADRC51-134', u'ADRC61-139', u'ADRC61-125', u'ADRC53-300', u'ADRC56-112', u'ADRC43-24', u'ADRC61-73', u'ADRC61-84', u'ADRC62-14', u'ADRC49-04', u'ADRC56-97', u'ADRC51-06', u'ADRC36-04', u'ADRC59-164', u'ADRC56-18', u'ADRC59-81', u'ADRC62-110', u'ADRC61-143', u'ADRC47-60', u'ADRC62-6', u'ADRC61-97', u'ADRC54-125', u'ADRC61-71', u'ADRC60-110', u'ADRC61-20', u'ADRC56-06', u'ADRC53-55', u'ADRC59-8', u'ADRC60-137', u'ADRC61-70', u'ADRC61-75', u'ADRC55-100', u'ADRC56-160', u'ADRC51-10', u'ADRC39-44', u'ADRC58-146', u'ADRC60-88', u'ADRC61-41', u'ADRC55-194', u'ADRC60-83', u'ADRC50-41', u'ADRC61-60', u'ADRC62-32', u'ADRC54-76', u'ADRC59-35', u'ADRC47-61', u'ADRC55-108', u'ADRC61-83', u'ADRC51-94', u'ADRC59-91', u'ADRC58-119', u'ADRC53-95', u'ADRC53-385', u'ADRC60-151', u'ADRC56-52', u'ADRC50-21', u'ADRC50-14

In [47]:
cur= dsa_slide_db['DSA_Slide_Data'].find({'pt_id':'ADRC50-10'})
for c in cur:
    print c

{u'slide_md5': None, u'slide_name': u'ADRC50-10_8-he.ndpi', u'width': 134912, u'height': 95488, u'pt_id': u'ADRC50-10', u'sld_properties': {u'hamamatsu_exposure_barcode_macro': u'320', u'openslide_level[3]_tile-width': u'496', u'openslide_level[6]_height': u'1492', u'hamamatsu_YOffsetFromSlideCentre': u'1636748', u'openslide_level[4]_tile-height': u'8', u'hamamatsu_PSHV_ploidy_10x': u'140', u'openslide_level[4]_tile-width': u'248', u'openslide_level[4]_height': u'5968', u'hamamatsu_Objective_Lens_Magnificant': u'35.16', u'hamamatsu_MHLN[1]_24bit': u'_1357FHEHqKOOTjx', u'openslide_level[1]_height': u'47744', u'openslide_level[4]_downsample': u'16', u'hamamatsu_slant_lefttop': u'91970,493576,112169', u'hamamatsu_colorfilterID': u'0', u'openslide_level[8]_downsample': u'256', u'openslide_level[2]_height': u'23872', u'tiff_DateTime': u'2013:07:25 22:48:43', u'openslide_level[2]_width': u'33728', u'hamamatsu_AHEX[0]': u'74270857AC1B7413083FAC0B741B043FAC0D7023043FA81170210847A40D6C43086BA83

In [34]:

## Going to create a cleanup and /or reformatted collection for the DSA Viewer
for s in dsa_slide_db['ADRC'].find():
    keys_of_interest = ['width','height']
    print s['slide_w_path']
    print s.keys()
    sys.exit()
###
